# Creacion de los archivos necesarios para el pipeline de bell

### El primer archivo con el que se va a trabajar es con TAIR10_3_utr_20101028.fasta

Se extraen todos los encabezados del fasta de la siguiente forma:

In [23]:
%%bash
cd ../Transcritos
grep '>' TAIR10_3_utr_20101028.fasta | sed -e 's/|[^|]*|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t'>  TAIR10_3_utr_20101028_encabezados.txt
head TAIR10_3_utr_20101028_encabezados.txt
cp TAIR10_3_utr_20101028_encabezados.txt ../GTF/

AT1G51370.2	chr1:19046749-19046825	FORWARD	77
AT1G50920.1	chr1:18872571-18872930	FORWARD	260
AT1G44020.1	chr1:16716112-16716691	REVERSE	580
AT1G15970.1	chr1:5486319-5486543	REVERSE	225
AT1G73440.1	chr1:27612183-27612815	FORWARD	633
AT1G75120.1	chr1:28196857-28197021	REVERSE	165
AT1G17600.1	chr1:6052936-6053025	REVERSE	90
AT1G51380.1	chr1:19049968-19050162	FORWARD	195
AT1G77370.1	chr1:29074643-29074796	FORWARD	154
AT1G10950.1	chr1:3663623-3663984	FORWARD	362


Ahora se extraen unicamente los 3 UTR del GTF:


In [24]:
%%bash
grep -v '#!\|##' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr' > GTF_three_prime_utr.txt
head GTF_three_prime_utr.txt

1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.1"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-203"; transcript_source "araport11"; transcript_biotype "protein_coding"; tag "Ensembl_canonical";
1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.4"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-202"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10943007	10943138	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-204"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10942663	10942909	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene

Como se puede apreciar en la columna 9 vienen varios parametros, por lo que se van a separar en diferentes columnas con el siguiente comando:

In [25]:
%%bash
awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}' GTF_three_prime_utr.txt | sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|awk -F'\t' 'BEGIN{OFS="\t"} {$1 = "chr" $1; print}'> GTF_three_prime_utr_final.txt
head GTF_three_prime_utr_final.txt

chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.1	AT1G30814	AT1G30814-203
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.4	AT1G30814	AT1G30814-202
chr1	araport11	three_prime_utr	10943007	10943138	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942663	10942909	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.3	AT1G30814	AT1G30814-201
chr1	araport11	three_prime_utr	29677904	29678284	.	-	.	AT1G78930	AT1G78930.1	AT1G78930	AT1G78930-201
chr1	araport11	three_prime_utr	26966558	26966688	.	+	.	AT1G71695	AT1G71695.1	AT1G71695	AT1G71695-201
chr1	araport11	three_prime_utr	21805932	21806278	.	-	.	AT1G58983	AT1G58983.1	AT1G58983	AT1G58983-201
chr1	araport11	three_prime_utr	4430778	4430965	.	+	.	AT1G12980	AT1G12980.1	ESR1	ESR1-201
chr1	araport11	three_prime_utr	17141620	17141771	.	+	.	AT1G45223	AT1G45223.1	AT1G45223	AT1G45223-201


Recordando que la columna 9 es **gene_id**, 10 es **transcript_id**, 11 es **gene_name** y 12 **transcript_name**

Teniendo estos dos archivos  
*TAIR10_3_utr_20101028_encabezados.txt*  
&  
*GTF_three_prime_utr_final.txt*  
  
Se va a llevar la comparacion linea por linea del fasta con el archivo del GTF para generar el archivo con las siguientes caracteristicas:

<table>
  <thead>
    <tr style="background-color: Turquoise; color: black;">
      <th>COL1</th>
      <th>COL2</th>
      <th>COL3</th>
      <th>COL4</th>
      <th>COL5</th>
      <th>COL6</th>
      <th>COL7</th>
      <th>COL8</th>
      <th>COL9</th>
      <th>COL10</th>
      <th>COL11</th>
      <th>COL12</th>
    </tr>
  <thead>
    <tr style="background-color: Aquamarine; color: black;">
      <th>Trancript_id</th>
      <th>Type</th>
      <th>Localization</th>
      <th>Chromosome</th>
      <th>Start</th>
      <th>End</th>
      <th>Strand</th>
      <th>Exon</th>
      <th>Gene_name</th>
      <th>Gene_id</th>
      <th>Gene_id2</th>
      <th>Length</th>
    </tr>
    <tr style="background-color: Aquamarine; color: black;">
      <th>COL1 -> TAIR10_3_utr</th>
      <th>COL3 -> GTF_three_prime</th>
      <th>COL2 -> TAIR10_3_utr</th>
      <th>COL1 -> GTF_three_prime</th>
      <th>COL4 -> GTF_three_prime</th>
      <th>COL5 -> GTF_three_prime</th>
      <th>COL7-> GTF_three_prime</th>
      <th>COL6 -> GTF_three_prime</th>
      <th>COL11 -> GTF_three_prime</th>
      <th>COL9 -> GTF_three_prime_utr</th>
      <th>COL9 -> GTF_three_prime_utr</th>
      <th>COL4 -> TAIR10_3_utr</th>
    </tr> 
       


Teniendo el contenido de la tabla se ejecuta el siguiente comando para generar el archivo 3' UTR:

In [26]:
%%bash
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt | head

AT1G30814.1 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10943007 10943138 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10942663 10942909 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.3 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G78930.1 three_prime_utr chr1:29678097-29678284 chr1 29677904 29678284 - . AT1G78930 AT1G78930 AT1G78930 188
AT1G71695.1 three_prime_utr chr1:26966558-26966688 chr1 26966558 26966688 + . AT1G71695 AT1G71695 AT1G71695 131
AT1G58983.1 three_prime_utr chr1:21806020-21806278 chr1 21805932 21806278 - . AT1G58983 AT1G58983 AT1G58983 259
AT1G12980.1 three_prime_utr chr1:4430778-4430965 chr1 4430778 4430965 + . ESR1 AT1G12980 AT1G12980 188
AT1G45223.1 three_prime_utr chr1:17141620-17141771 chr1 17141620 17141771 + . AT1G45223 AT1G45223 AT1G45223 152
A

Se ve bien, ahora lo unico que me queda es guardar el archivo.

In [27]:
%%bash
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt > ../coords_dict/TAIR10_3_UTR.coords.dict